In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from pydataset import data

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Acquire

In [2]:
#load mpg data
df = data('mpg')
df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


# Prepare

In [3]:
# rename reserved words
df = df.rename(columns={'class':'body_style'})

In [4]:
#encode trans column for machine learning
df.trans = np.where(df.trans.str.startswith('auto'), 0, 1)

In [5]:
df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,body_style
1,audi,a4,1.8,1999,4,0,f,18,29,p,compact
2,audi,a4,1.8,1999,4,1,f,21,29,p,compact
3,audi,a4,2.0,2008,4,1,f,20,31,p,compact
4,audi,a4,2.0,2008,4,0,f,21,30,p,compact
5,audi,a4,2.8,1999,6,0,f,16,26,p,compact


# Model

In [6]:
#split into X and y
X = df[['displ', 'year', 'cyl', 'cty', 'hwy', 'trans']]
X.head()

,displ,year,cyl,cty,hwy,trans
1,1.8,1999,4,18,29,0
2,1.8,1999,4,21,29,1
3,2.0,2008,4,20,31,1
4,2.0,2008,4,21,30,0
5,2.8,1999,6,16,26,0


In [7]:
# check out target
y = df.body_style
y.head()

1    compact
2    compact
3    compact
4    compact
5    compact
Name: body_style, dtype: object

In [29]:
df.body_style.value_counts(normalize=True).head(1)

suv    0.264957
Name: body_style, dtype: float64

In [9]:
#import
from sklearn.model_selection import cross_val_score
#import
from sklearn.model_selection import GridSearchCV

In [10]:
#split into trains and tests
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=123)

In [11]:
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)

X_train: (175, 6)
X_test: (59, 6)


# KNN

In [30]:
y_train.value_counts()

suv           41
compact       36
midsize       34
subcompact    29
pickup        24
minivan        9
2seater        2
Name: body_style, dtype: int64

In [42]:
#make our modeling object
knn = KNeighborsClassifier()

In [43]:
#define hyperparameters to send in
parameters= {'n_neighbors': range(1,21)}
parameters

{'n_neighbors': range(1, 21)}

In [44]:
#make the gridsearch object
grid = GridSearchCV(knn, parameters, cv=5)
grid

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)})

In [45]:
#fit the gridsearch object
grid.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciP

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. 

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)})

In [16]:
#look at mean test scores
grid.cv_results_['mean_test_score']

array([0.46857143, 0.4       , 0.45142857, 0.45142857, 0.45714286,
       0.48      , 0.46285714, 0.43428571, 0.44      , 0.44      ,
       0.42857143, 0.41142857, 0.40571429, 0.41714286, 0.42857143,
       0.42285714, 0.41714286, 0.42857143, 0.42857143, 0.41714286])

In [17]:
#what is the best score
grid.best_score_

0.47999999999999987

In [18]:
#what is the best estimator
grid.best_estimator_

KNeighborsClassifier(n_neighbors=6)

In [19]:
# what are the predictions
grid.predict(X_test)

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


array(['midsize', 'pickup', 'compact', 'midsize', 'pickup', 'pickup',
       'compact', 'suv', 'minivan', 'suv', 'minivan', 'midsize',
       'minivan', 'suv', 'minivan', 'pickup', 'pickup', 'suv', 'midsize',
       'compact', 'compact', 'compact', 'pickup', 'suv', 'suv', 'compact',
       'compact', 'compact', 'subcompact', 'suv', 'pickup', 'midsize',
       'suv', 'suv', 'pickup', 'midsize', 'subcompact', 'subcompact',
       'suv', 'pickup', 'subcompact', 'pickup', 'suv', 'minivan',
       'midsize', 'suv', 'suv', 'compact', 'pickup', 'compact', '2seater',
       'midsize', 'compact', 'subcompact', 'subcompact', 'pickup', 'suv',
       'midsize', 'subcompact'], dtype=object)

In [20]:
# what is the probability of those predictions
grid.predict_proba(X_test)[:10]

array([[0.        , 0.16666667, 0.66666667, 0.        , 0.        ,
        0.16666667, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 0.5       ],
       [0.        , 0.5       , 0.16666667, 0.        , 0.        ,
        0.33333333, 0.        ],
       [0.        , 0.16666667, 0.83333333, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.        , 0.66666667,
        0.        , 0.33333333],
       [0.        , 0.5       , 0.        , 0.        , 0.        ,
        0.5       , 0.        ],
       [0.        , 0.        , 0.        , 0.16666667, 0.33333333,
        0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       , 0.        ,
        0.33333333, 0.16666667],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.16666667, 0.8

In [21]:
#save into varible to run on test data
knn_model=grid.best_estimator_
knn_model

KNeighborsClassifier(n_neighbors=6)

In [22]:
#score model on test data
knn_model.score(X_test, y_test)

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.3898305084745763

# Cross validate

In [23]:
#split into trains and tests
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=123)

In [24]:
#make it
tree = DecisionTreeClassifier(max_depth=4, random_state=123)

In [25]:
#fit and score
cross_val_score(tree, X_train, y_train, cv=5)

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


array([0.48571429, 0.54285714, 0.57142857, 0.51428571, 0.42857143])

In [26]:
cross_val_score(tree, X_train, y_train, cv=5).mean()

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.5085714285714285